## Importing Libraries and setting up the client

In [23]:
import pymongo 
from pymongo import MongoClient
import praw
from praw.models import MoreComments
import pandas as pd
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from cred import login
client = MongoClient('mongodb://127.0.0.1:27017')
db = client['reddit']

In [6]:
reddit = praw.Reddit(client_id =login['client_id'],client_secret =login['client_secret'],user_agent = login['user_agent'])
subreddit = reddit.subreddit('india')

### Scraping Reddit Data and Saving it to MongoDB 

In [12]:
#list of all the flairs, these will be keys in classification task

flairs = ["AskIndia", "Non-Political", "[R]eddiquette", 
          "Scheduled", "Photography", "Science/Technology",
          "Politics", "Business/Finance", "Policy/Economy",
          "Sports", "Food", "AMA"]

for flair in flairs:
    
    relevants = subreddit.search(flair,limit=100)
    for post in relevants:
        posts ={
        "title":str(post.title),
        "score":str(post.score),
        "id":str(post.id),
        "url":str(post.url),
        "comms_num":str(post.num_comments),
        "created":str(post.created),
        "body":str(post.selftext),
        "author":str(post.author),
        "flair":str(flair),	
        }
        
#since some post have a lot of comments , therefore only top 10 comments and their authors will be considered

        post.comments.replace_more(limit=None)
        comment =""
        authors = ""
        count =10
        for top_comment in post.comments:
            print(count)
            comment = comment+' '+top_comment.body
            authors = authors+' '+str(top_comment.author)
            count+=1
            if(count>10):
                break
        posts['comment'] = str(comment)
        posts['authors'] = str(authors)
        res = db.posts.insert_one(posts)

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
1

In [16]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /home/dy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:

replace_by_space = re.compile('[/(){}\[\]\|@,;]')
replace_symbol = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_cleaner(text):
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = replace_by_space.sub(' ', text) # replace certain symbols by space in text
    text = replace_symbol.sub('', text) # delete symbols from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove STOPWORDS from text
    return text



In [22]:


collection = db.posts
posts = pd.DataFrame(list(collection.find()))

del posts['_id']

posts['title'] = posts['title']
posts['body'] = posts['body']
posts['comment'] = posts['comment']

posts['title'] = posts['title'].apply(text_cleaner)
posts['body'] = posts['body'].apply(text_cleaner)
posts['comment'] = posts['comment'].apply(text_cleaner)

combined_features = posts["title"] + posts["comment"] + posts["url"] + posts["body"]
posts = posts.assign(combined_features = combined_features)

#saving the csv file
posts.to_csv('data.csv',index=False)

